In [1]:
import ee


In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWhtoqsgpejF23nVhEAbuOKixHcX2_TzUiFWyJ6I3qkW1eg162-hE2o

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
import geemap
import ipyleaflet
import numpy as np

In [5]:
## specify areas of interest / districts
Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
#print(Districts.limit(1).getInfo())

In [6]:
## create map
Map = geemap.Map(height="300px")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [7]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)

In [8]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();
Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

In [9]:
## add tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
TreePctThreshold = 10 #whole numbers - 0-100

## Reproject trees to match LULC projection
WCprojection = WC.first().projection(); 
print('Trees projection:', WCprojection.getInfo());
TreeCover = TreeCover.reproject(
      crs= WCprojection
    )

Map.addLayer(TreeCover.updateMask(TreeCover.gte(TreePctThreshold)),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)

Trees projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}


In [10]:
#add area of interest to map
Map.addLayer(Districts,{},"Districts",True,0.5)


In [11]:
## calculations to determine tree cover by LULC class



asClass = WorldCover
toCount = TreeCover

## function to create image of means of toCount for each asClass

def getmeanbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment if you want includ only pixel that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('') #'class_count-'
                                       .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass
def getcountbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment if you want includ only pixel that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_count-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))


In [12]:
## create image with each class mean as a band

meanbyclass=ee.Image(getmeanbyclass(10)).addBands([
  getmeanbyclass(20),  
  getmeanbyclass(30),  
  getmeanbyclass(40),  
  getmeanbyclass(50),  
  getmeanbyclass(60),
  getmeanbyclass(70),  
    getmeanbyclass(80), 
    getmeanbyclass(90), 
    getmeanbyclass(95), 
    getmeanbyclass(100), 
])

## create image with each class count as a band

countbyclass=ee.Image(getcountbyclass(10)).addBands([
  getcountbyclass(20),  
  getcountbyclass(30),  
  getcountbyclass(40),  
  getcountbyclass(50),  
  getcountbyclass(60),
  getcountbyclass(70),  
    getcountbyclass(80), 
    getcountbyclass(90), 
    getcountbyclass(95), 
    getcountbyclass(100), 
])

#print('meanbyclass', meanbyclass.getInfo())
#print('countbyclass', countbyclass.getInfo())
Map.addLayer(meanbyclass.select('50'),{},"meanbyclass")

In [13]:
## create FeatureCollection with mean of count for each class for each feature

histo=meanbyclass.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclass.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

print('histo:', histo.limit(1).getInfo())

histo: {'type': 'FeatureCollection', 'columns': {'10': 'Float<0.0, 255.0>', '100': 'Float<0.0, 255.0>', '20': 'Float<0.0, 255.0>', '30': 'Float<0.0, 255.0>', '40': 'Float<0.0, 255.0>', '50': 'Float<0.0, 255.0>', '60': 'Float<0.0, 255.0>', '70': 'Float<0.0, 255.0>', '80': 'Float<0.0, 255.0>', '90': 'Float<0.0, 255.0>', '95': 'Float<0.0, 255.0>', 'Area_ha': 'Float', 'Id': 'Integer', 'Sub_City': 'String', 'Woreda': 'String', 'area_m2': 'Float', 'class_count-10': 'Long<0, 4294967295>', 'class_count-100': 'Long<0, 4294967295>', 'class_count-20': 'Long<0, 4294967295>', 'class_count-30': 'Long<0, 4294967295>', 'class_count-40': 'Long<0, 4294967295>', 'class_count-50': 'Long<0, 4294967295>', 'class_count-60': 'Long<0, 4294967295>', 'class_count-70': 'Long<0, 4294967295>', 'class_count-80': 'Long<0, 4294967295>', 'class_count-90': 'Long<0, 4294967295>', 'class_count-95': 'Long<0, 4294967295>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinat

In [14]:
## Define function to normalize count as percent of all pixels in each feature and create new properties with the values

def count_to_percent(feat):
    feat=ee.Feature(feat)
    hist=ee.Dictionary(feat.toDictionary(['10','20','30','40','50','60','70','80','90','95','100']))
    hist=hist.set('10',hist.get('10',0))
    hist=hist.set('20',hist.get('20',0))
    hist=hist.set('30',hist.get('30',0))
    hist=hist.set('40',hist.get('40',0))
    hist=hist.set('50',hist.get('50',0))
    hist=hist.set('60',hist.get('60',0))
    hist=hist.set('70',hist.get('70',0))
    hist=hist.set('80',hist.get('80',0))
    hist=hist.set('90',hist.get('90',0))
    hist=hist.set('95',hist.get('95',0))
    hist=hist.set('100',hist.get('100',0))
    
    def pct_hist(k,v):
        # convert to whole number (0-100) to decimal percent (0-1)
        return ee.Number(v).multiply(ee.Number(0.01))
    pcts = hist.map(pct_hist)
    
    histC=ee.Dictionary(feat.toDictionary(['class_count-10','class_count-20','class_count-30','class_count-40','class_count-50','class_count-60','class_count-70','class_count-80','class_count-90','class_count-95','class_count-100']))
    histC=histC.set('10',hist.get('10',0))
    histC=histC.set('20',hist.get('20',0))
    histC=histC.set('30',hist.get('30',0))
    histC=histC.set('40',hist.get('40',0))
    histC=histC.set('50',hist.get('50',0))
    histC=histC.set('60',hist.get('60',0))
    histC=histC.set('70',hist.get('70',0))
    histC=histC.set('80',hist.get('80',0))
    histC=histC.set('90',hist.get('90',0))
    histC=histC.set('95',hist.get('95',0))
    histC=histC.set('100',hist.get('100',0))
 
    area=feat.area().multiply(0.000001)

    totalPixels=hist.values()
    
    def area_hist(k,v):
        # convert 10m pixel count of class to KM2 of class
        return ee.Number(v).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    classAreas = histC.map(area_hist)
    
    return feat.set({
        'LC10treepct': pcts.getNumber('10'),
        'LC20treepct': pcts.getNumber('20'),
        'LC30treepct': pcts.getNumber('30'),
        'LC40treepct': pcts.getNumber('40'),
        'LC50treepct': pcts.getNumber('50'),
        'LC60treepct': pcts.getNumber('60'),
        'LC70treepct': pcts.getNumber('70'),
        'LC80treepct': pcts.getNumber('80'),
        'LC90treepct': pcts.getNumber('90'),
        'LC95treepct': pcts.getNumber('95'),
        'LC100treepct': pcts.getNumber('100'),
        'TotalareaKM2': area,
        'TotalPixels': totalPixels,
        'LC10areaKM2': classAreas.getNumber('10'),
        'LC20areaKM2': classAreas.getNumber('20'),
        'LC30areaKM2': classAreas.getNumber('30'),
        'LC40areaKM2': classAreas.getNumber('40'),
        'LC50areaKM2': classAreas.getNumber('50'),
        'LC60areaKM2': classAreas.getNumber('60'),
        'LC70areaKM2': classAreas.getNumber('70'),
        'LC80areaKM2': classAreas.getNumber('80'),
        'LC90areaKM2': classAreas.getNumber('90'),
        'LC95areaKM2': classAreas.getNumber('95'),
        'LC100areaKM2': classAreas.getNumber('100')
    })

In [15]:
## update FeatureCollection with percents

tree_pcts=histo.map(count_to_percent)

print('Tree cover % by Land Cover class for Districts',tree_pcts.limit(1).getInfo());

Tree cover % by Land Cover class for Districts {'type': 'FeatureCollection', 'columns': {'10': 'Float<0.0, 255.0>', '100': 'Float<0.0, 255.0>', '20': 'Float<0.0, 255.0>', '30': 'Float<0.0, 255.0>', '40': 'Float<0.0, 255.0>', '50': 'Float<0.0, 255.0>', '60': 'Float<0.0, 255.0>', '70': 'Float<0.0, 255.0>', '80': 'Float<0.0, 255.0>', '90': 'Float<0.0, 255.0>', '95': 'Float<0.0, 255.0>', 'Area_ha': 'Float', 'Id': 'Integer', 'LC100areaKM2': 'Number', 'LC100treepct': 'Number', 'LC10areaKM2': 'Number', 'LC10treepct': 'Number', 'LC20areaKM2': 'Number', 'LC20treepct': 'Number', 'LC30areaKM2': 'Number', 'LC30treepct': 'Number', 'LC40areaKM2': 'Number', 'LC40treepct': 'Number', 'LC50areaKM2': 'Number', 'LC50treepct': 'Number', 'LC60areaKM2': 'Number', 'LC60treepct': 'Number', 'LC70areaKM2': 'Number', 'LC70treepct': 'Number', 'LC80areaKM2': 'Number', 'LC80treepct': 'Number', 'LC90areaKM2': 'Number', 'LC90treepct': 'Number', 'LC95areaKM2': 'Number', 'LC95treepct': 'Number', 'Sub_City': 'String', 'T

In [16]:
## render on map percent tree cover by class from feature collection

empty = ee.Image().byte()
Tpctfills = empty.paint(**{'featureCollection': tree_pcts,'color': 'LC50treepct'})

fillspalette = ['red', 'green']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':0.15}, '% of built-up area with tree cover', True, 0.65)
Map

Map(center=[8.98091786068599, 38.78520904303285], controls=(WidgetControl(options=['position', 'transparent_bg…

In [17]:
## select properties to keep, sort features and create data frame to display properties
tree_pctsSort = tree_pcts.select(['Woreda','Sub_City','LC10treepct','LC20treepct','LC30treepct','LC40treepct','LC50treepct','LC60treepct','LC70treepct','LC80treepct','LC90treepct','LC95treepct','LC100treepct','LC10areaKM2','LC20areaKM2','LC30areaKM2','LC40areaKM2','LC50areaKM2','LC60areaKM2','LC70areaKM2','LC80areaKM2','LC90areaKM2','LC95areaKM2','LC100areaKM2']).sort('LC50treepct', False)
print('Tree cover sorted version',tree_pctsSort.limit(1).getInfo());



Tree cover sorted version {'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[38.753431897333755, 9.076388717555735], [38.75346313006271, 9.076317371755659], [38.75348541632305, 9.07625491308227], [38.753494362776664, 9.076219283681464], [38.75353446204468, 9.076125655644459], [38.75357456128797, 9.076063238630505], [38.753596847510096, 9.075991843334904], [38.75360132072939, 9.07591160724839], [38.75360132072939, 9.075849185819314], [38.753619213603464, 9.075719837372958], [38.75365036636355, 9.075679713267851], [38.753677125773756, 9.075612846454634], [38.75372177801877, 9.075452311035209], [38.75372177801877, 9.075372016330428], [38.753739590980956, 9.075331910529847], [38.753744064190414, 9.07530962773158], [38.753739590980956, 9.075300700141511], [38.75372177801877, 9.075269448242208], [38.75370388517294, 9.075238275076511], [38.753708358384856, 9.075202586982297], [38.753726171350706, 9.075175817886588], 

In [18]:
df = geemap.ee_to_pandas(tree_pctsSort)
df

,LC90areaKM2,LC95treepct,LC20treepct,LC10treepct,LC100treepct,LC40treepct,LC50treepct,LC95areaKM2,LC60treepct,LC40areaKM2,...,LC70treepct,LC70areaKM2,LC90treepct,Sub_City,LC20areaKM2,LC80areaKM2,LC80treepct,LC100areaKM2,LC30treepct,LC10areaKM2
0,0.008,0,0.739248,0.871967,0,0.102089,0.118593,0,0.186512,0.001021,...,0,0,0.8,Gulele,0.007392,0,0,0,0.440925,0.008720
1,0.000,0,0.488394,0.756258,0,0.282876,0.107469,0,0.144502,0.002829,...,0,0,0.0,Gulele,0.004884,0,0,0,0.193649,0.007563
2,0.000,0,0.522543,0.754897,0,0.284533,0.104440,0,0.151427,0.002845,...,0,0,0.0,Nifassilk Lafto,0.005225,0,0,0,0.236128,0.007549
3,0.000,0,0.767672,0.851176,0,0.238987,0.104337,0,0.100421,0.002390,...,0,0,0.0,Nifassilk Lafto,0.007677,0,0,0,0.296262,0.008512
4,0.000,0,0.432064,0.741092,0,0.283020,0.094070,0,0.148487,0.002830,...,0,0,0.0,Lideta,0.004321,0,0,0,0.123346,0.007411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.000,0,0.461103,0.814784,0,0.123428,0.017397,0,0.035585,0.001234,...,0,0,0.0,Nifassilk Lafto,0.004611,0,0,0,0.123940,0.008148
112,0.000,0,0.000000,0.620618,0,0.710135,0.016454,0,0.016678,0.007101,...,0,0,0.0,Lideta,0.000000,0,0,0,0.000000,0.006206
113,0.000,0,0.000000,0.706325,0,0.000000,0.013937,0,0.020942,0.000000,...,0,0,0.0,Addis Ketema,0.000000,0,0,0,0.029487,0.007063
114,0.000,0,0.000000,0.682236,0,0.076744,0.008455,0,0.012251,0.000767,...,0,0,0.0,Addis Ketema,0.000000,0,0,0,0.000000,0.006822


In [19]:
## display features in chart

import geemap.chart as chart

xProperty = "Woreda"
yProperties = ['LC50treepct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "% Tree Cover",
    "legend_location": "top-right",
    "height": "500px",
}

chart.feature_byFeature(tree_pctsSort, xProperty, yProperties, **options)

In [20]:
#df.groupby(df.properties).apply(lambda x: np.average(x.LC50treepct, weights=x.LC50areaKM2))

def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()

weighted_average(df, 'LC50treepct', 'LC50areaKM2')

0.0630297430701035

In [21]:
# Download attribute table as a CSV

url = tree_pctsSort.getDownloadURL(
    filetype="csv",
    #selectors=['NAME', 'ALAND', 'REGION', 'STATEFP', 'STUSPS'],
    filename="tree_pctsSort",
)
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/156bb0373aa170476969b1c389f1ee6a-0b521c6460dba30aadff06db1ae8412f:getFeatures


In [ ]:
# Download data as shapefile to Google Drive

# function for converting GeometryCollection to Polygon/MultiPolygon
def filter_polygons(ftr):
    geometries = ftr.geometry().geometries()
    geometries = geometries.map(
        lambda geo: ee.Feature(ee.Geometry(geo)).set('geoType', ee.Geometry(geo).type())
    )

    polygons = (
        ee.FeatureCollection(geometries)
        .filter(ee.Filter.eq('geoType', 'Polygon'))
        .geometry()
    )
    return ee.Feature(polygons).copyProperties(ftr)

tree_pctsSortSHP = tree_pctsSort.map(filter_polygons)

col_names = tree_pctsSort.first().propertyNames().getInfo()
print("Column names: ", col_names)

url = tree_pctsSortSHP.getDownloadURL("shp", col_names, 'tree_pctsSortSHP')
print(url)

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'SHP',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}

print('Exporting {}'.format(tree_pctsSort))
task = ee.batch.Export.table.toDrive(tree_pctsSort, 'AddisTreePctsbyLULC', **task_config)
task.start()

Column names:  ['system:index', 'LC90areaKM2', 'LC95treepct', 'LC20treepct', 'LC10treepct', 'LC100treepct', 'LC40treepct', 'LC50treepct', 'LC95areaKM2', 'LC60treepct', 'LC40areaKM2', 'Woreda', 'LC50areaKM2', 'LC60areaKM2', 'LC30areaKM2', 'LC70treepct', 'LC70areaKM2', 'LC90treepct', 'Sub_City', 'LC20areaKM2', 'LC80areaKM2', 'LC80treepct', 'LC100areaKM2', 'LC30treepct', 'LC10areaKM2']
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/793819f8f2c531bec8b74c76926c4a23-01fd38b3be5d0cd7f2dab3b50b86dae2:getFeatures


In [ ]:
# Download ee.Image for Land cover and tree cover as GeoTIFF

geemap.ee_export_image(
    WorldCover, filename="ESAWorldCover.tif", scale=10, region=Districts.geometry(), file_per_band=False
)

geemap.ee_export_image(
    TreeCover, filename="TMLTreeCover.tif", scale=10, region=Districts.geometry(), file_per_band=False
)

In [ ]:
# OR if filesize is too large for standard download - Export an ee.Image to Google Drive as TIF

geemap.ee_export_image_to_drive(
    TreeCover, description='TMLTreeCover', folder='EEexport', region=Districts.geometry(), scale=10
)